In [1]:
import arcpy

arcpy.env.overwriteOutput = True
#miejsce gdzie beda sie zapisywaly warstwy posrednie #smieci z ktorych beda kopiowane punkty do interesujących nas warstw
gdb_route =  r"D:\MDokumenty\studia\GotliboweLov\wektoryzacja\przasnysz\ud_projekt_PrzasnyszBudynekOBLOT\ud_projekt_PrzasnyszBudynekOBLOT.gdb"
arcpy.env.workspace = gdb_route
##tutaj true jesli chce sie zeby sie dodawaly warstwy posrednie do projektu
arcpy.env.addOutputsToMap = False

Zaleca się przed uruchomieniem algorytmu:
--ZROBIĆ BUCKUP
-- WYCZYŚCIĆ JAKIEKOLWIEK SELEKCJE
-- WSZYSTKIE PIETRA WLACZYC NA RAZ (inaczej wiekszosc funkcji zadziala tylko dla biezacego widoku)

In [2]:
#WCZYTANIE TABELI FLOOR
floor_table = arcpy.da.TableToNumPyArray("FLOOR", ["ID", "BASE_ELEVATION"])
floor_IDs = [i[0] for i in floor_table]
floor_dic = {}
for id, elev in floor_table:
    floor_dic[id] = elev


In [3]:
#funkcja dla każdej wprowadzonej warstwy, iteruje się po wszystkich piętrach, 
#podczas tej iteracji zamienia obiekty z warstwy na punkty zapisujac w oddzielnej warstwie śmieciowej, ale z ID_FLOOR :)
#dlatego koniecznie trzeba zmienic sciezke dostepu przy zapisie Feature to point w funkcji na ud_GmachCzegos.gdb z) bo powstaje tyle roboczych warstw co pieter
#nastepnie przekopiowuje te elementy do klas state i route node, 

def create_staty_nody(warstwa): #przyjmuje stringa z nazwa warstwy np, "ROOM", ""
    for floor in floor_IDs:
        print("robi sie:", warstwa, "pietro", floor)
        where = warstwa + ".FLOOR_ID =" + str(floor)   
        file_name = warstwa +"_point" + str(floor)
        roomy_floor = arcpy.management.SelectLayerByAttribute(r"topografia\\" + warstwa, "NEW_SELECTION", where) 
        room_point = arcpy.management.FeatureToPoint(r"topografia\\" + warstwa, gdb_route+ "\\" + file_name , "INSIDE")
        arcpy.management.CalculateField(file_name, "FLOOR_ID", str(floor), "PYTHON3", '', "TEXT")
        edit = arcpy.da.Editor(arcpy.env.workspace)
        edit.startEditing(True)
        edit.startOperation()
        arcpy.Append_management(file_name, r"nawigacja\State", "NO_TEST") #wklejanie
        arcpy.Append_management(file_name, r"nawigacja\RouteNode", "NO_TEST")
        edit.stopOperation()
        edit.stopEditing(True)
        arcpy.management.SelectLayerByAttribute(r"topografia\\" + warstwa, "REMOVE_FROM_SELECTION", where)
#warto zauwazyc za zawsze state zrobiony dla danej warstwy z nizszego pietra bedzie mial mniejsze objectid niz ten z wyzszego

In [159]:
def open_raport(budynek):
    file = open(f'D:\\raport_{budynek}.txt', 'r')
    to_print = ''
    for line in file:
        splitted = line.split(' ')
        if splitted[0] == 'warstwa':
            if to_print != '':
                print(f"{to_print[:-1]})')" )
            to_print = f"z0_in_warstwa('{splitted[1]}','("
        if splitted[0] == 'OID:':
            to_print = f"{to_print}{splitted[1]}"
    print(f"{to_print[:-1]})')" )
    

In [160]:
open_raport('GmachMatematyki')

z0_in_warstwa('ROOM
','(7,14,25,137,183,184,185,186,191,201,204,208,209,217,219,235,236,237,238,239,240,241,242,243,244,246,247,253,265,273,277,282,284,286,287,288,289,290,293,294,295,309,318,319,320,323,330,331,332,333,334,442,444,455,464,465,466,469,476,493,495,496,506)')
z0_in_warstwa('ROOF_SURFACE
','(3)')
z0_in_warstwa('INTERIORWALL_SURFACE
','(477,478,2108,2111,2113)')
z0_in_warstwa('CEILING_SURFACE
','(22,23,24,29,35,61,68,70,72,83,84,92,94,105,124,130,138,140,142,143,154,155,156,157,158,159,160,161,162,163,164,167,171,186,188,189,213,215,226,235,236,237,240,247,248,283,285,296,305,306,307,310,317,353,354,355,358,359,360,373,382,383,384,387,394,395,396,397,398,423)')
z0_in_warstwa('BUILDING_INSTALLATION
','(3)')


In [5]:
#funkcja do updatowania Z na 0 w danej warstwie
def z0_in_warstwa(warstwa, IDS):
    warstwa_long = r"topografia\\" + warstwa
    arcpy.management.RemoveJoin(warstwa_long)
    edit = arcpy.da.Editor(arcpy.env.workspace)
    edit.startEditing(True)
    edit.startOperation()
    where = f'{warstwa}.OBJECTID IN {IDS}'
    with  arcpy.da.UpdateCursor(warstwa_long,  ['OBJECTID', 'SHAPE@'], where) as cursor:
        for row in cursor:
            
            if row[1].isMultipart == False:
                tablica = arcpy.Array()
                for point in row[1].getPart()[0]:
                    new = point
                    new.Z = 0
                    tablica.append(new)
                row[1] = arcpy.Polygon(tablica, has_z = True)
                cursor.updateRow(row)      
                print(row[0])
            else: 
                array = arcpy.Array()
                for part in row[1].getPart():
                    tablica = arcpy.Array()
                    for point in row[1].getPart()[0]:
                        try:
                            new = point
                            new.Z = 0
                            tablica.append(new)
                        except AttributeError: #punkt None
                            tablica.append(tablica[0]) #u mnie nie powinno generować bledow, bo wiem jak rysowalam i dlaczego sa te none punkty
                    array.append(tablica)
                row[1] = arcpy.Polygon(array, has_z = True)
                cursor.updateRow(row)      
                print(row[0])
                    
    edit.stopOperation()
    edit.stopEditing(True)
    arcpy.management.AddJoin(warstwa_long, "FLOOR_ID", "FLOOR", "ID")

In [213]:
z0_in_warstwa('CEILING_SURFACE','(22,23,24,29,35,61,68,70,72,83,84,92,94,105,124,130,138,140,142,143,154,155,156,157,158,159,160,161,162,163,164,167,171,186,188,189,213,215,226,235,236,237,240,247,248,283,285,296,305,306,307,310,317,353,354,355,358,359,360,373,382,383,384,387,394,395,396,397,398,423)')

error
error
error
22
23
24
29
35
61
error
68
70
72
error
83
84
error
92
error
94
error
105
124
error
error
130
138
140
142
143
error
154
error
error
155
error
156
error
157
error
158
error
error
159
error
160
error
161
error
162
error
163
error
164
167
171
186
188
189
error
213
215
error
226
error
235
236
error
237
error
240
247
error
error
248
error
283
285
error
296
error
305
306
error
307
error
310
317
353
354
355
358
359
error
360
error
373
error
382
383
error
384
error
387
394
error
error
395
396
397
398
error
error
error
423


In [138]:
#funkcja do updatowania Z na 0 w danej warstwie
def z0_in_warstwa(warstwa):
    warstwa_long = r"topografia\\" + warstwa
    edit = arcpy.da.Editor(arcpy.env.workspace)
    edit.startEditing(True)
    edit.startOperation()
    with  arcpy.da.UpdateCursor(warstwa_long,  ['OBJECTID', 'SHAPE@']) as cursor:
        for row in cursor:
            tablica = arcpy.Array()
            for point in row[1].getPart()[0]:
                new = point
                new.Z = 0
                tablica.append(new)
            row[1] = arcpy.Polygon(tablica, has_z = True)
            cursor.updateRow(row)      
            print(row[0])
            for point in row[1].getPart()[0]:
                print(point.Z)
    edit.stopOperation()
    edit.stopEditing(True)

In [139]:
z0_in_warstwa('ROOM')

<class 'arcpy.arcobjects.arcobjects.Array'>
7500888,2533 5787625,9783 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500887,5833 5787626,0407 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500886,3925 5787626,1663 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500885,8643 5787626,2203 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500886,2423 5787630,2785 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500886,4037 5787630,2635 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500886,4837 5787631,1309 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500888,3527 5787630,9455 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500888,2697 5787630,0889 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500888,6337 5787630,0537 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500888,6035 5787629,7259 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500888,4673 5787628,2693 0 NaN
<class 'arcpy.arcobjects.arcobjects.Array'>
7500888,2533 5787625,9783 0 NaN
0.0
0.0
0.0


0
1474
0
1475


In [6]:
#funkcja do uruchomienia po funkcji saty_nody w celu ustawienie Z dla RouteNodów - nie dziala
#iteruje się po RouteNodach i ustawia ich Z na BaseElevation danego piętra + 1 m
def z_in_windows():
    edit = arcpy.da.Editor(arcpy.env.workspace)
    edit.startEditing(True)
    edit.startOperation()
    with  arcpy.da.SearchCursor(r"topografia\WINDOW",  ['OBJECTID', 'FLOOR_ID', 'ELEVATION_OVERBASE']) as window_cursor:
        for window_row in window_cursor:
            where_window = f'OBJECTID = {window_row[0]}'
            windows = arcpy.management.SelectLayerByAttribute(r"topografia\WINDOW", "NEW_SELECTION", where_window)
            arcpy.management.SelectLayerByLocation(r"nawigacja\RouteNode", "INTERSECT", r"topografia\WINDOW" )
            where_node = f'RouteNode.FLOOR_ID ={window_row[1]}' 
            with arcpy.da.UpdateCursor(r"nawigacja\RouteNode",  ['OBJECTID', 'FLOOR_ID', 'SHAPE@Z'], where_node) as cursor:
                for row in cursor:
                    row[2] = 1+floor_dic[window_row[1]]+window_row[2]
                    cursor.updateRow(row)
                arcpy.management.SelectLayerByAttribute(r"nawigacja\RouteNode", "REMOVE_FROM_SELECTION", where_node)
            arcpy.management.SelectLayerByAttribute(r"topografia\WINDOW", "REMOVE_FROM_SELECTION", where_window)
    edit.stopOperation()
    edit.stopEditing(True)

In [4]:
#funkcja do uruchomienia po funkcji saty_nody w celu ustawienie Z dla RouteNodów w oknach
#iteruje się po RouteNodach i ustawia ich Z na BaseElevation danego piętra + 1 m
def z_in_nodes():
    edit = arcpy.da.Editor(arcpy.env.workspace)
    edit.startEditing(True)
    edit.startOperation()
    for floor in floor_table:
        ID = floor[0]
        elev = floor[1]
        print(ID, elev)
        where =  "RouteNode.FLOOR_ID =" + str(ID)   
        nody_floor = arcpy.management.SelectLayerByAttribute(r"nawigacja\RouteNode", "NEW_SELECTION", where) 
        with arcpy.da.UpdateCursor(r"nawigacja\RouteNode",  ['OBJECTID', 'FLOOR_ID', 'SHAPE@Z']) as cursor:
            for row in cursor:
                row[2] = elev+1
                cursor.updateRow(row)
        arcpy.management.SelectLayerByAttribute(r"nawigacja\RouteNode", "REMOVE_FROM_SELECTION", where)
    edit.stopOperation()
    edit.stopEditing(True)

In [6]:
#create_staty_nody("ROOM")
#create_staty_nody("DOOR")
#create_staty_nody("WINDOW")
create_staty_nody("BUILDING_INSTALLATION")

#Przypisuje STATE_ID w klasie RouteNode, jako OBJECTID - bo jak kopiujemy to równocześnie do obu tabel 
#i wykonujemy operacje na  pustych tabelach State i RouteNode to wartości OBJECTID te są identyczne dla odpowiadajacych statów i nodow w obu klasach
#arcpy.management.CalculateField(r"nawigacja\RouteNode", "STATE_ID", "!OBJECTID!", "PYTHON3", '', "TEXT")

z_in_nodes()

robi sie: BUILDING_INSTALLATION pietro 1
robi sie: BUILDING_INSTALLATION pietro 2
robi sie: BUILDING_INSTALLATION pietro 3
robi sie: BUILDING_INSTALLATION pietro 4
1 -2.56
2 0.0
3 3.25
4 6.51


In [22]:
z_in_windows()

In [6]:
create_staty_nody("BUILDING_INSTALATION")

robi sie: BUILDING_INSTALATION pietro 1


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Layer Name or Table View: Dataset topografia\\BUILDING_INSTALATION does not exist or is not supported
Failed to execute (SelectLayerByAttribute).


In [10]:
#A tu oddzielnie bo sobie puszczalam ta funkcje po dodaniu nowych nodow #polecam jak np sie zapomni zmienic wartosci w z mode 
z_in_nodes()

1 0.0
2 3.24
3 6.54
4 9.14


In [3]:
#funkcja staty_nody_from_to iteruje sie po obiektach liniowych znajduje intersekcje z obiektami punktowymi drugiej warstwy 
#i wpisuje object id obiektu punktowego do do odpowiedniego miejsca w tabeli obiektu liniowego

def moj_cursor(points, lines, points_fields, row):
    first_point = row[4].firstPoint
    last_point = row[4].lastPoint
    #print(cursor)
    where = "OBJECTID = " + str(row[0])
    arcpy.management.SelectLayerByAttribute(lines, "NEW_SELECTION", where) 
    arcpy.management.SelectLayerByLocation(points, "INTERSECT", lines)
    with arcpy.da.SearchCursor(points, points_fields) as point_cursor:
        for point in point_cursor:
            if points == r"nawigacja\State":  
                if point[1] == row[1] and point[2] == first_point:
                    row[2] = point[0]
                elif point[1] == row[1] and point[2] == last_point:
                    row[3] = point[0]
            elif points == r"nawigacja\RouteNode":
                if point[2] == first_point:
                    row[2] = point[0]
                elif point[2] == last_point:
                    row[3] = point[0]
        if row[2] == 0 or row[3] == 0:
            with arcpy.da.SearchCursor(points, points_fields) as point_cursor:
                for point in point_cursor:
                    if point[1] == (row[1] + 1) and point[2] == first_point:
                        row[2] = point[0]
                    elif point[1] == (row[1] + 1) and point[2] == last_point:
                        row[3] = point[0]
        if row[2] == 0 or row[3] == 0:
            print("ŹLE DOSNAPOWANA LINIA!!! ---> OBJECTID = ", row[0], "FLOOR_ID =", row[1])
            blad = True
            
def staty_nody_from_to(lines, points, fields_lines, points_fields, overWrite = True):   
    arcpy.management.RemoveJoin(lines)
    arcpy.management.RemoveJoin(points)
    arcpy.management.SelectLayerByAttribute(lines, "CLEAR_SELECTION") 
    arcpy.management.SelectLayerByAttribute(points, "CLEAR_SELECTION")
    edit = arcpy.da.Editor(arcpy.env.workspace)
    edit.startEditing(True)
    edit.startOperation()
    blad = False
    if overWrite:        
        with arcpy.da.UpdateCursor(lines,  fields_lines) as cursor:
            for row in cursor:
                moj_cursor(points, lines, points_fields, row)
                cursor.updateRow(row)
    else:
        where = str(fields_lines[2]) +"= 0 Or " + str(fields_lines[3]) +"= 0"
        with arcpy.da.UpdateCursor(lines,  fields_lines, where) as cursor:
            for row in cursor:
                moj_cursor(points, lines, points_fields, row)
                cursor.updateRow(row)            
    arcpy.management.SelectLayerByAttribute(lines, "CLEAR_SELECTION") 
    arcpy.management.SelectLayerByAttribute(points, "CLEAR_SELECTION")
    edit.stopOperation()      
    edit.stopEditing(True)
    arcpy.management.AddJoin(lines, "FLOOR_ID", "FLOOR", "ID")
    arcpy.management.AddJoin(points, "FLOOR_ID", "FLOOR", "ID")
    if blad:
        print("istnieje też prawdopodobienstwo, ze Maria o czyms nie pomyslala, wiec spojrz gdzie to i przekaz informacje autorowi ")
#uwaga funkcja moze troche nie dzialac na schodach w momencie gdy staty, albo routeNody są idealnie nad sobą na roznych pietrach

In [4]:
#wywolanie funkcji znajdujaca state_from i state_to dla transition
transitions  = r"nawigacja\Transition"
states = r"nawigacja\State"
transitions_fields = ['OBJECTID', 'FLOOR_ID', 'STATE_FROM_ID', 'STATE_TO_ID', 'SHAPE@']
state_fields = ['OBJECTID', 'FLOOR_ID', 'SHAPE@']
staty_nody_from_to(transitions, states, transitions_fields, state_fields, False) 
#ostatni parametr określa czy funkcja nadpisuje wartości, jeśli false, to tylko tam gdzie nie ma wartości wpisanych w state from i state to je wpisze
#analogicznie we wszystkich pozostalych

In [7]:
#wywolanie funkcji znajdujaca routenode_from i routenode_to dla routesegment
route_nody =  r"nawigacja\RouteNode"
route_segmenty = r"nawigacja\RouteSegment"
routeSegments_fields = ['OBJECTID', 'FLOOR_ID', 'ROUTENODE_FROM_ID', 'ROUTENODE_TO_ID', 'SHAPE@']
routeNodes_fields = ['OBJECTID', 'FLOOR_ID', 'SHAPE@']
staty_nody_from_to(route_segmenty, route_nody, routeSegments_fields, routeNodes_fields, False)

In [23]:
#funkcja staty_ids znajduje staty które przecinają się zdaną warstwą i są na tym samym piętrze i wpisuje objectid state do jego "STATE_ID"
states = r"nawigacja\State"
def moj_cursor_state_id(warstwa, row):
    #print(cursor)
    where = "OBJECTID = " + str(row[0])
    arcpy.management.SelectLayerByAttribute(warstwa, "NEW_SELECTION", where) 
    arcpy.management.SelectLayerByLocation(states, "INTERSECT", warstwa)
    with arcpy.da.SearchCursor(states,"OBJECTID","State.FLOOR_ID=" +str(row[1]) ) as point_cursor:
        for point in point_cursor: 
            row[2] = point[0]
    if row[2] == 0:
        print("nie znalazlem state_id dla roomu o objectid = ", row[0])

            
def staty_ids(warstwa,  overWrite = True): 
    warstwa_1 = r"topografia\\" + warstwa
    arcpy.management.RemoveJoin(warstwa_1)
    arcpy.management.SelectLayerByAttribute(warstwa_1, "CLEAR_SELECTION") #
    arcpy.management.SelectLayerByAttribute(states, "CLEAR_SELECTION")#
    edit = arcpy.da.Editor(arcpy.env.workspace)
    edit.startEditing(True)
    edit.startOperation()
    if overWrite:        
        with arcpy.da.UpdateCursor(warstwa_1,  ["OBJECTID", "FLOOR_ID", "STATE_ID"]) as cursor:
            for row in cursor:
                moj_cursor_state_id(warstwa_1, row)
                cursor.updateRow(row)
    else:
        where = "STATE_ID IS NULL"
        with arcpy.da.UpdateCursor(warstwa_1,  ["OBJECTID", "FLOOR_ID", "STATE_ID"], where) as cursor:
            for row in cursor:
                moj_cursor_state_id(warstwa_1, row)
                cursor.updateRow(row)            
    edit.stopOperation()      
    edit.stopEditing(True)
    arcpy.management.AddJoin(warstwa_1, "FLOOR_ID", "FLOOR", "ID")
    arcpy.management.SelectLayerByAttribute(warstwa_1, "CLEAR_SELECTION") #
    arcpy.management.SelectLayerByAttribute(states, "CLEAR_SELECTION") #
    print("Done State_ID w", warstwa)

In [24]:
#przypisanie wartosci state id dla roomow
staty_ids("ROOM", False)
#powinno działac tak jak wyżej, bez iterowania sie po tej tabeli ale cos nie gra, wiec robie selekcje po pietrach i wtedy wywoluje funkcje na masce juz
#jak chcecie tak sprobowac to w funckji wyzej trzeba zakomentowac te linijki ktore maja na koncu hasza
"""
warstwa = "ROOM"
for floor in floor_IDs:
    where = warstwa + ".FLOOR_ID =" + str(floor) 
    warstwa_1 = r"topografia\\" + warstwa
    arcpy.management.SelectLayerByAttribute(warstwa_1, "NEW_SELECTION", where)
    arcpy.management.SelectLayerByAttribute(states, "NEW_SELECTION", where)
    staty_ids("ROOM", False)
    arcpy.management.SelectLayerByAttribute(warstwa_1, "REMOVE_FROM_SELECTION", where)
    arcpy.management.SelectLayerByAttribute(states, "REMOVE_FROM_SELECTION", where)
"""


Done State_ID w ROOM


'\nwarstwa = "ROOM"\nfor floor in floor_IDs:\n    where = warstwa + ".FLOOR_ID =" + str(floor) \n    warstwa_1 = r"topografia\\" + warstwa\n    arcpy.management.SelectLayerByAttribute(warstwa_1, "NEW_SELECTION", where)\n    arcpy.management.SelectLayerByAttribute(states, "NEW_SELECTION", where)\n    staty_ids("ROOM", False)\n    arcpy.management.SelectLayerByAttribute(warstwa_1, "REMOVE_FROM_SELECTION", where)\n    arcpy.management.SelectLayerByAttribute(states, "REMOVE_FROM_SELECTION", where)\n'

In [25]:
#wpisanie state id do drzwi - to samo mozna dla okien jak ktos ma to trzeba odkomentowac;P
staty_ids("DOOR", False) #tutaj zadzialalo przy wylaczonej filtracji por que, no se
staty_ids("WINDOW", False)

Done State_ID w DOOR
Done State_ID w WINDOW


In [8]:
#funkcja make_staty)_ids_in_nodes znajduje intersekcje routeNodow z roomami i przepisuje STATE_ID z roomu do routNoda

def moj_cursor_state_id_2(warstwa, rooms, row):
    #print(cursor)
    rooms_top =  r"topografia\\" + rooms
    where = "OBJECTID = " + str(row[0])
    arcpy.management.SelectLayerByAttribute(warstwa, "NEW_SELECTION", where) 
    arcpy.management.SelectLayerByLocation(rooms_top, "INTERSECT", warstwa)
    with arcpy.da.SearchCursor(rooms_top,f"{rooms}.STATE_ID",f"{rooms}.FLOOR_ID=" +str(row[1]) ) as room_cursor:
        for room in room_cursor: 
            row[2] = room[0]

            
def make_staty_ids_in_nodes(rooms, overWrite = True):
    rooms_top = r"topografia\\" + rooms
    warstwa = "RouteNode"
    warstwa_1 = r"nawigacja\\" + warstwa
    
    arcpy.management.RemoveJoin(warstwa_1)
    arcpy.management.SelectLayerByAttribute(warstwa_1, "CLEAR_SELECTION") 
    arcpy.management.SelectLayerByAttribute(rooms_top, "CLEAR_SELECTION")
    edit = arcpy.da.Editor(arcpy.env.workspace)
    edit.startEditing(True)
    edit.startOperation()
    if overWrite:        
        with arcpy.da.UpdateCursor(warstwa_1,  ["OBJECTID", "FLOOR_ID", "STATE_ID"]) as cursor:
            for row in cursor:
                moj_cursor_state_id_2(warstwa_1, rooms, row)
                cursor.updateRow(row)
    else:
        where = "STATE_ID = 0 "
        with arcpy.da.UpdateCursor(warstwa_1,  ["OBJECTID", "FLOOR_ID", "STATE_ID"], where) as cursor:
            for row in cursor:
                moj_cursor_state_id_2(warstwa_1,rooms, row)
                cursor.updateRow(row)            
    arcpy.management.SelectLayerByAttribute(warstwa_1, "CLEAR_SELECTION") 
    arcpy.management.SelectLayerByAttribute(rooms_top, "CLEAR_SELECTION")
    edit.stopOperation()      
    edit.stopEditing(True)
    arcpy.management.AddJoin(warstwa_1, "FLOOR_ID", "FLOOR", "ID")
    print(f"Done State_ID w {warstwa} dla {rooms}")

In [11]:
make_staty_ids_in_nodes(r"ROOM", False)
#make_staty_ids_in_nodes(r"WINDOW",True)
#make_staty_ids_in_nodes(r"DOOR",True)

Done State_ID w RouteNode dla ROOM
